In [1]:
import numpy as np # library to handle data in a vectorized manner
import keras 
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from sklearn import metrics

import numpy as np
from keras.layers import Dense, Activation, regularizers
from keras.models import Sequential
from keras import initializers
from keras.regularizers import l2
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout

import seaborn as sn


from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Function for turning a vector into a one-hot representation

In [2]:
def one_hot(a,C):
    Yoh = np.zeros((a.shape[0],C))
    size = a.shape[0]
    i = 0
    while i < size:
        if(a.iloc[i]==1):
            Yoh[i] = [1,0,0]
        if(a.iloc[i]==2):
            Yoh[i] = [0,1,0]
        if(a.iloc[i]==3):
            Yoh[i] = [0,0,1] 
        #if(a.iloc[i]==4):
        #    Yoh[i] = [0,0,0,1]
        #if(a.iloc[i]==5):
        #    Yoh[i] = [0,0,0,0,1]
        i = i + 1    
    return Yoh 

Function for turning a softmax representation into a binary one

In [3]:
def roundup(a):
    Yoh = np.zeros((a.shape[0],a.shape[1]))
    size = a.shape[0]
    i = 0
    while i < size:
        if(np.argmax(a[i])==0):
            Yoh[i] = [1,0,0]
        if(np.argmax(a[i])==1):
            Yoh[i] = [0,1,0]
        if(np.argmax(a[i])==2):
            Yoh[i] = [0,0,1] 
        #if(np.argmax(a[i])==3):
        #    Yoh[i] = [0,0,0,1]
        #if(np.argmax(a[i])==4):
        #    Yoh[i] = [0,0,0,0,1]
        i = i + 1    
    return Yoh 

Error Computation

In [4]:
def errorComp(a,b):
    size = a.shape[0]
    i = 0
    error = 0
    while i < size:
        if(np.argmax(a[i]) != np.argmax(b[i])):
            error = error + 1
        i = i + 1    
    return error/size*100 

Classifying the severity of an engine based on the remaining useful life count

In [5]:
def categorization(a,L1,inc):
    i = 0
    while i < a.shape[0]:
        if(a.loc[i,"RUL"]<(L1+1)):
            a.loc[i,"Class"] = 1
        elif(a.loc[i,"RUL"]<(L1+inc+1)):
            a.loc[i,"Class"] = 2
        elif(a.loc[i,"RUL"]>(L1 + inc)):
            a.loc[i,"Class"] = 3
        #elif(a.loc[i,"RUL"] > (L1+2*inc)):
        #    a.loc[i,"Class"] = 4
        #elif(a.loc[i,"RUL"] > (L1+3*inc)):
        #    a.loc[i,"Class"] = 5
        i = i+1
    return a

In [6]:
Master = pd.read_excel("Master v1.xlsx")
Master['Class'] = ""
Master.head()

,Opp Setting 1,Opp Setting 2,Opp Setting 3,SM2,SM3,SM4,SM7,SM9,SM11,SM12,SM14,SM15,SM17,SM20,SM21,RUL,Class
0,0.0023,0.0003,100,643.02,1585.29,1398.21,553.90,9050.17,47.20,521.72,8125.55,8.4052,392,38.86,23.3735,142,
1,-0.0027,-0.0003,100,641.71,1588.45,1395.42,554.85,9054.42,47.50,522.16,8139.62,8.3803,393,39.02,23.3916,141,
2,0.0003,0.0001,100,642.46,1586.94,1401.34,554.11,9056.96,47.50,521.97,8130.10,8.4441,393,39.08,23.4166,140,
3,0.0042,0.0000,100,642.44,1584.12,1406.42,554.07,9045.29,47.28,521.38,8132.90,8.3917,391,39.00,23.3737,139,
4,0.0014,0.0000,100,642.51,1587.19,1401.92,554.16,9044.55,47.31,522.15,8129.54,8.4031,390,38.99,23.4130,138,


Computing the correlation matrix to determine which features can be dropped

In [7]:
corrMatrix = Master.corr()
sn.heatmap(corrMatrix, annot=True, fmt='g')
plt.figure(figsize=(40,40))
plt.show()

In [8]:
Master.drop(['Opp Setting 1','Opp Setting 2','Opp Setting 3','SM7','SM12','SM21','SM20'],axis = 1,inplace = True)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

Creating the training and test sets which will feed into a simple dense neural network composed of one hidden layer, 32 activations and 4 softmax output

In [ ]:
L1 = 40
inc = 30
TestResults = []
TrainResults = []
Ranges = []
i = 0
cats = 3
Samples = Master.shape[0]
TestS = 3000
TrainS = Samples - TestS
#for i in range(60):
Master = categorization(Master,L1,inc)
Raw = shuffle(Master)
Raw.reset_index(inplace=True)
    
Y = Raw.loc[:,'Class']
X = Raw.iloc[:,1:9]
    
Xtest = X.iloc[0:TestS,:]
Ytest = Y.iloc[0:TestS]
Xtrain = X.iloc[TestS:,:]
Ytrain = Y.iloc[TestS:]
    
Xtrain.reset_index(drop=True, inplace=True)
Ytrain.reset_index(drop=True, inplace=True)
Xtest.reset_index(drop=True, inplace=True)
Ytest.reset_index(drop=True, inplace=True)
    
YtrainOH = one_hot(Ytrain,cats)
YtestOH = one_hot(Ytest,cats)
    
Xtrain = sc.fit_transform(Xtrain)
Xtest = sc.fit_transform(Xtest)
    
    
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 8,kernel_initializer=initializers.RandomNormal(stddev=0.001),bias_initializer=initializers.Zeros()))
model.add(Dense(cats, activation='softmax'))
optimizer = keras.optimizers.Adam(lr = 0.01)
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy',metrics=['accuracy'])
model.fit(Xtrain, YtrainOH, batch_size = 128, epochs = 10)
    
y_pred = model.predict(Xtest)
y_predR = roundup(y_pred)
y_pred2 = model.predict(Xtrain)
y_pred2R = roundup(y_pred2)
Trainacc = metrics.accuracy_score(YtrainOH, y_pred2R)
Testacc = metrics.accuracy_score(YtestOH,y_predR)
    
print((i+1)*10," Epochs\n")
print(L1,"L1\n")
print("Train set Accuracy: ", Trainacc)
print("Test set Accuracy: ", Testacc)
    
Ranges.append(L1)
TestResults.append(Trainacc)
TrainResults.append(Testacc)
L1 = L1 + 1

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Epoch 1/10
72043/72043 [==============================] - 7s 103us/step - loss: 0.4837 - acc: 0.8272
Epoch 2/10
72043/72043 [==============================] - 7s 96us/step - loss: 0.4056 - acc: 0.8467: 1s - loss: 
Epoch 3/10
72043/72043 [==============================] - 7s 99us/step - loss: 0.3979 - acc: 0.8503
Epoch 4/10
72043/72043 [==============================] - 7s 95us/step - loss: 0.3931 - acc: 0.8529
Epoch 5/10
72043/72043 [==============================] - 7s 97us/step - loss: 0.3911 - acc: 0.8534
Epoch 6/10
72043/72043 [==============================] - 7s 91us/step - loss: 0.3904 - acc: 0.8535
Epoch 7/10
72043/72043 [==============================] - 7s 93us/step - loss: 0.3897 - acc: 0.8538
Epoch 8/10
72043/72043 [==============================] - 7s 93us/step - loss: 0.3894 - acc: 0.8540
Epoch 9/10
72043/72043 [==============================] - 7s 94us/step - loss: 0.3888 - acc: 0.8538
Epoch 10/10
72043/72043 [==============================] - 7s 97us/step - loss: 0.3886

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Epoch 1/10
72043/72043 [==============================] - 7s 100us/step - loss: 0.5125 - acc: 0.8259
Epoch 2/10
72043/72043 [==============================] - 7s 97us/step - loss: 0.4048 - acc: 0.8483: 0s - loss: 0.4051 - a
Epoch 3/10
72043/72043 [==============================] - 7s 95us/step - loss: 0.3965 - acc: 0.8518: 0s - loss: 0.395
Epoch 4/10
72043/72043 [==============================] - 7s 100us/step - loss: 0.3944 - acc: 0.8523
Epoch 5/10
72043/72043 [==============================] - 7s 100us/step - loss: 0.3931 - acc: 0.8524
Epoch 6/10
72043/72043 [==============================] - 7s 97us/step - loss: 0.3921 - acc: 0.8530
Epoch 7/10
72043/72043 [==============================] - 8s 111us/step - loss: 0.3918 - acc: 0.8529
Epoch 8/10
72043/72043 [==============================] - 7s 92us/step - loss: 0.3914 - acc: 0.8529
Epoch 9/10
72043/72043 [==============================] - 7s 97us/step - loss: 0.3911 - acc: 0.8526
Epoch 10/10
72043/72043 [==============================

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


...

Epoch 1/10
72043/72043 [==============================] - 9s 125us/step - loss: 0.5286 - acc: 0.8074 2s - loss: 0.5497 - acc: 0.802 - ETA: 2
Epoch 2/10
72043/72043 [==============================] - 7s 103us/step - loss: 0.4668 - acc: 0.8245
Epoch 3/10
72043/72043 [==============================] - 7s 95us/step - loss: 0.4595 - acc: 0.8278
Epoch 4/10
72043/72043 [==============================] - 7s 93us/step - loss: 0.4571 - acc: 0.8287
Epoch 5/10
72043/72043 [==============================] - 8s 106us/step - loss: 0.4552 - acc: 0.8290
Epoch 6/10
72043/72043 [==============================] - 7s 103us/step - loss: 0.4542 - acc: 0.8291
Epoch 7/10
72043/72043 [==============================] - 7s 95us/step - loss: 0.4534 - acc: 0.8291
Epoch 8/10
72043/72043 [==============================] - 7s 100us/step - loss: 0.4529 - acc: 0.8294
Epoch 9/10
72043/72043 [==============================] - 8s 117us/step - loss: 0.4526 - acc: 0.8297 0s - loss: 0.4521 - a
Epoch 10/10
72043/72043 [=======

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Epoch 1/10
72043/72043 [==============================] - 8s 109us/step - loss: 0.5454 - acc: 0.8024
Epoch 2/10
72043/72043 [==============================] - 7s 93us/step - loss: 0.4705 - acc: 0.8230
Epoch 3/10
72043/72043 [==============================] - 7s 94us/step - loss: 0.4657 - acc: 0.8244
Epoch 4/10
72043/72043 [==============================] - 8s 116us/step - loss: 0.4631 - acc: 0.8256
Epoch 5/10
72043/72043 [==============================] - 7s 103us/step - loss: 0.4618 - acc: 0.8259
Epoch 6/10
72043/72043 [==============================] - 7s 103us/step - loss: 0.4606 - acc: 0.8264
Epoch 7/10
72043/72043 [==============================] - 8s 107us/step - loss: 0.4601 - acc: 0.8264
Epoch 8/10
72043/72043 [==============================] - 7s 97us/step - loss: 0.4594 - acc: 0.8262
Epoch 9/10
72043/72043 [==============================] - 7s 98us/step - loss: 0.4591 - acc: 0.8260
Epoch 10/10
72043/72043 [==============================] - 8s 107us/step - loss: 0.4588 - acc: 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Epoch 1/10
72043/72043 [==============================] - 8s 113us/step - loss: 0.5490 - acc: 0.8007
Epoch 2/10
72043/72043 [==============================] - 7s 100us/step - loss: 0.4780 - acc: 0.8199
Epoch 3/10
72043/72043 [==============================] - 7s 103us/step - loss: 0.4728 - acc: 0.8227
Epoch 4/10
72043/72043 [==============================] - 7s 96us/step - loss: 0.4703 - acc: 0.8230
Epoch 5/10
50560/72043 [====================>.........] - ETA: 2s - loss: 0.4700 - acc: 0.8227